In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
# load data
df = pd.read_csv("../data/final_merged_data.csv", index_col=0)

In [5]:
# Keep Ticker for reference but exclude it from inputs
tickers = df["Ticker"]  # Store tickers for later use
df_numeric = df.drop(columns=["Date", "Ticker"], errors="ignore")  # Drop non-numeric

# Handle missing values by filling with column mean
df_numeric.fillna(df_numeric.mean(), inplace=True)

# Define features and target
X = df_numeric.drop(columns=["Risk_Premium"])
y = df_numeric["Risk_Premium"]

# Train-test split
X_train, X_test, y_train, y_test, tickers_train, tickers_test = train_test_split(
    X, y, tickers, test_size=0.2, random_state=42
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define and train the neural network
mlp = MLPRegressor(hidden_layer_sizes=(64, 32, 16), activation='relu', solver='adam', 
                   max_iter=500, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = mlp.predict(X_train_scaled)
y_test_pred = mlp.predict(X_test_scaled)

# Evaluation Metrics
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

print(f"Train R²: {train_r2:.4f}, Train RMSE: {train_rmse:.4f}")
print(f"Test R²: {test_r2:.4f}, Test RMSE: {test_rmse:.4f}")

# Store predictions with tickers
predictions_df = pd.DataFrame({
    "Ticker": tickers_test,
    "Actual_Risk_Premium": y_test,
    "Predicted_Risk_Premium": y_test_pred
})

Train R²: 0.3360, Train RMSE: 0.1587
Test R²: -0.0479, Test RMSE: 0.1852


c:\Users\yangt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
c:\Users\yangt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\yangt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
# Save predictions
predictions_df.to_csv("neural_network_predictions.csv", index=False)